In [1]:
# import prices from a csv file
import pandas as pd
prices = pd.read_csv("data/price.csv", index_col=0, header=0, parse_dates=True)

In [2]:
# Connect with MongoDB (here using a mock database)
# Here you could go wild and point to an existing MongoDB Server. 
from mongoengine import *
connect(db="test", host='mongomock://localhost', alias='default')

mongomock.MongoClient('localhost', 27017)

In [3]:
# Symbols are arranged in groups. Hence we need a group first
from pyutil.mongo.engine.symbol import Symbol, Group
g = Group(name="US Stock").save()

# if you get a duplicate error with your MongoMock database just restart the kernel

In [4]:
# Create a bunch of symbols
for key, data in prices.items():
    symbol = Symbol(name=key, group=g, internal=key, price=data.dropna()).save()

In [5]:
# Loop over all symbols to do flips and shit
for symbol in Symbol.objects:
    print(symbol.name)
    print(symbol.group.name)
    print(symbol.internal)
    print(symbol.price.tail(3))

A
US Stock
A
2015-04-20    1195.88
2015-04-21    1202.34
2015-04-22    1200.59
Name: A, dtype: float64
B
US Stock
B
2015-04-20    27094.93
2015-04-21    27850.49
2015-04-22    27964.84
Name: B, dtype: float64
C
US Stock
C
2015-04-16    54674.21
2015-04-17    53954.79
2015-04-20    53761.27
Name: C, dtype: float64
D
US Stock
D
2015-04-20    2667.63
2015-04-21    2651.41
2015-04-22    2638.95
Name: D, dtype: float64
E
US Stock
E
2015-04-17    2081.18
2015-04-20    2100.40
2015-04-21    2097.29
Name: E, dtype: float64
F
US Stock
F
2015-04-20    3718.04
2015-04-21    3719.38
2015-04-22    3742.77
Name: F, dtype: float64
G
US Stock
G
2015-04-17    3061.38
2015-04-20    3059.73
2015-04-21    3065.53
Name: G, dtype: float64


In [9]:
# Fish for a particular symbol and create some reference data
s1 = Symbol.objects(name="A").get()
print(s1.name)
print(s1.group.name)
s1.reference["AA"] = 20.0
s1.reference["BB"] = 30.0
s1.save()

s2 = Symbol.objects(name="C").get()
print(s2.name)
print(s2.group.name)
s2.reference["BB"] = 40.0
s2.reference["CC"] = 30.0
s2.save()

A
US Stock
C
US Stock


<Symbol: <Symbol: C>>

In [10]:
# Construct a frame with all reference data for a bunch of symbols
Symbol.reference_frame(products=[s1,s2])

,AA,BB,CC,Sector,Internal
symbol,,,,,
A,20.0,30.0,NaN,US Stock,A
C,NaN,40.0,30.0,US Stock,C


In [14]:
# and again but for all symbols
Symbol.reference_frame()

,AA,BB,CC,Sector,Internal
symbol,,,,,
A,20.0,30.0,NaN,US Stock,A
B,NaN,NaN,NaN,US Stock,B
C,NaN,40.0,30.0,US Stock,C
D,NaN,NaN,NaN,US Stock,D
E,NaN,NaN,NaN,US Stock,E
F,NaN,NaN,NaN,US Stock,F
G,NaN,NaN,NaN,US Stock,G


In [ ]:
# extract a frame of time series data
Symbol.pandas_frame(item="price", products=[s1,s2])

In [ ]:
# and again but for all symbols
Symbol.pandas_frame(item="price")

In [ ]:
# and again but for all symbols
Symbol.pandas_frame(item="price")

In [ ]:
# and again but for all symbols
Symbol.pandas_frame(item="price")

symbol,A,C
2013-01-01,1673.78,62550.10
2013-01-02,1686.90,62550.10
2013-01-03,1663.95,63312.46
2013-01-04,1655.65,62523.06
2013-01-07,1646.95,61932.54
...,...,...
2015-04-16,1198.56,54674.21
2015-04-17,1204.27,53954.79
2015-04-20,1195.88,53761.27
2015-04-21,1202.34,NaN


In [12]:
# and again but for all symbols
Symbol.pandas_frame(item="price")

symbol,A,B,C,D,E,F,G
2013-01-01,1673.78,23311.98,62550.10,3735.12,1462.42,2711.25,2518.99
2013-01-02,1686.90,23311.98,62550.10,3735.12,1462.42,2711.25,2518.99
2013-01-03,1663.95,23398.60,63312.46,3714.99,1459.37,2701.22,2509.51
2013-01-04,1655.65,23331.09,62523.06,3689.34,1466.47,2709.35,2516.81
2013-01-07,1646.95,23329.75,61932.54,3699.14,1461.89,2695.56,2523.77
...,...,...,...,...,...,...,...
2015-04-16,1198.56,27739.71,54674.21,2694.50,2104.99,3751.72,3077.21
2015-04-17,1204.27,27653.12,53954.79,2676.61,2081.18,3674.05,3061.38
2015-04-20,1195.88,27094.93,53761.27,2667.63,2100.40,3718.04,3059.73
2015-04-21,1202.34,27850.49,NaN,2651.41,2097.29,3719.38,3065.53
